<a href="https://colab.research.google.com/github/Shaeeraaa/NNDL/blob/main/face_recognition_attendance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://www.cl.cam.ac.uk/Research/DTG/attarchive/pub/data/att_faces.zip


--2026-02-06 04:00:27--  https://www.cl.cam.ac.uk/Research/DTG/attarchive/pub/data/att_faces.zip
Resolving www.cl.cam.ac.uk (www.cl.cam.ac.uk)... 128.232.0.20, 2a05:b400:110::80:14
Connecting to www.cl.cam.ac.uk (www.cl.cam.ac.uk)|128.232.0.20|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cl.cam.ac.uk/research/dtg/attarchive/pub/data/att_faces.zip [following]
--2026-02-06 04:00:28--  https://www.cl.cam.ac.uk/research/dtg/attarchive/pub/data/att_faces.zip
Reusing existing connection to www.cl.cam.ac.uk:443.
HTTP request sent, awaiting response... 200 OK
Length: 3769022 (3.6M) [application/zip]
Saving to: ‘att_faces.zip’

att_faces.zip       100%[===================>]   3.59M  3.29MB/s    in 1.1s    

2026-02-06 04:00:29 (3.29 MB/s) - ‘att_faces.zip’ saved [3769022/3769022]



In [ ]:
!unzip att_faces.zip


Archive:  att_faces.zip
  inflating: README                  
   creating: s1/
  inflating: s1/6.pgm                
  inflating: s1/7.pgm                
  inflating: s1/8.pgm                
  inflating: s1/9.pgm                
  inflating: s1/10.pgm               
  inflating: s1/1.pgm                
  inflating: s1/2.pgm                
  inflating: s1/3.pgm                
  inflating: s1/4.pgm                
  inflating: s1/5.pgm                
   creating: s2/
  inflating: s2/6.pgm                
  inflating: s2/7.pgm                
  inflating: s2/8.pgm                
  inflating: s2/9.pgm                
  inflating: s2/10.pgm               
  inflating: s2/1.pgm                
  inflating: s2/2.pgm                
  inflating: s2/3.pgm                
  inflating: s2/4.pgm                
  inflating: s2/5.pgm                
   creating: s3/
  inflating: s3/6.pgm                
  inflating: s3/7.pgm                
  inflating: s3/8.pgm                
  inflating: 

In [ ]:
!mv att_faces faces_dataset


mv: cannot stat 'att_faces': No such file or directory


In [ ]:
!mkdir faces_dataset


In [ ]:
!mv s* faces_dataset/


In [ ]:
!ls faces_dataset


s1   s12  s15  s18  s20  s23  s26  s29	s31  s34  s37  s4   s6	s9
s10  s13  s16  s19  s21  s24  s27  s3	s32  s35  s38  s40  s7	sample_data
s11  s14  s17  s2   s22  s25  s28  s30	s33  s36  s39  s5   s8


In [ ]:
# Install required libraries
!pip install opencv-python tensorflow
# Imports
import os
import cv2
import numpy as np
import pandas as pd
from datetime import datetime
import tensorflow as tf
from tensorflow.keras import layers, models
from google.colab import files
# Dataset path
DATASET_PATH = "/content/faces_dataset"
IMG_SIZE = 112   # AT&T images are 92x112
# Load dataset
X = []
y = []
labels = {}
label_id = 0
# Use only first 5 people (faster + accurate)
for person in sorted(os.listdir(DATASET_PATH))[:5]:
    labels[label_id] = person
    person_path = os.path.join(DATASET_PATH, person)
    for img_name in os.listdir(person_path):
        img_path = os.path.join(person_path, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img / 255.0
        X.append(img)
        y.append(label_id)
    label_id += 1
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = tf.keras.utils.to_categorical(y)
print("Dataset loaded:", X.shape)
# CNN Model
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(labels), activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
# Train
model.fit(X, y, epochs=6, batch_size=8)
# Attendance dataframe
attendance = pd.DataFrame(columns=["Name", "Time"])
def mark_attendance(name):
    if name not in attendance["Name"].values:
        attendance.loc[len(attendance)] = [
            name,
            datetime.now().strftime("%H:%M:%S")]
# Upload test image
print("Upload a test image (from s1, s2, etc.)")
uploaded = files.upload()
test_img = list(uploaded.keys())[0]
# Predict
img = cv2.imread(test_img, cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
img = img / 255.0
img = np.expand_dims(img, axis=(0, -1))
pred = model.predict(img)
name = labels[np.argmax(pred)]
mark_attendance(name)
print("Recognized student:", name)
# Save attendance
attendance.to_csv("attendance.csv", index=False)
attendance

Dataset loaded: (50, 112, 112, 1)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/6
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 275ms/step - accuracy: 0.3459 - loss: 1.9146
Epoch 2/6
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6135 - loss: 1.4398 
Epoch 3/6
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8298 - loss: 1.0143 
Epoch 4/6
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9369 - loss: 0.4865 
Epoch 5/6
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9567 - loss: 0.2027 
Epoch 6/6
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0468 
Upload a test image (from s1, s2, etc.)


Saving Alexandra Daddario_41.jpg to Alexandra Daddario_41.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 517ms/step
Recognized student: s1


,Name,Time
0,s1,04:05:50


wildlife
